In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Ridge
import xgboost as xgb
from lightgbm import LGBMRegressor
from scipy.stats import randint, uniform
from matplotlib.backends.backend_pdf import PdfPages
import chardet  # 用于检测文件编码

# 打印 XGBoost 和 LightGBM 版本
print(f"XGBoost version: {xgb.__version__}")

# 1. 加载数据集
dataset_path = r"D:\data\data03.csv"

# 检测文件编码
with open(dataset_path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

print(f"Detected encoding: {encoding}")

# 使用检测到的编码读取文件
dataset = pd.read_csv(dataset_path, engine='python', encoding=encoding)

# 2. 数据质量检查：处理异常值或噪声
X = dataset.iloc[:, 0:11]  # 保留 Pandas DataFrame 格式，便于后续分析
y = dataset.iloc[:, 19]

# 将目标变量乘以 10^9 以放大数值范围
y = y * 1e9

# 检查目标变量的异常值（例如，通过 IQR 方法）
Q1 = y.quantile(0.25)
Q3 = y.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 移除目标变量的异常值
filtered_dataset = dataset[(y >= lower_bound) & (y <= upper_bound)]
X = filtered_dataset.iloc[:, 0:11]
y = filtered_dataset.iloc[:, 19] * 1e9  # 确保目标变量也进行了放大

# 3. 特征工程：特征重要性分析
# 使用 XGBoost 的默认参数训练模型，获取特征重要性
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X, y)

# 获取特征重要性
feature_importances = xgb_model.feature_importances_
feature_names = X.columns

# 特征名缩写（确保特殊字符显示正常）
abbreviated_names = {
    "2θ/°": "2Theta",
    "Intensity": "Inten",
    "Peak Width": "PWidth",
    "Peak Height": "PHeight",
    "Background": "Bkg",
    "Lattice Parameter": "LParam",
    "Crystallite Size": "CSize",
    "Strain": "Strain",
    "Phase Composition": "PhaseComp",
    "Surface Area": "SurfArea",
    "Porosity": "Porosity"
}

# 将特征名替换为缩写
abbreviated_feature_names = [abbreviated_names.get(name, name) for name in feature_names]

# 可视化特征重要性
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=abbreviated_feature_names)
plt.title("Feature Importances")
plt.tight_layout()  # 调整布局以确保特征名完整显示

# 4. 数据拆分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 模型训练和超参数调优
def train_model_with_tuning(model, param_grid, X_train, y_train, cv=5, n_iter=50):
    tuner = RandomizedSearchCV(model, param_grid, n_iter=n_iter, cv=cv, n_jobs=-1, random_state=42, verbose=2)
    tuner.fit(X_train, y_train)
    best_model = tuner.best_estimator_
    print(f"Best parameters: {tuner.best_params_}")
    return best_model

# 调整后的超参数网格
param_grid_lgb = {
    'n_estimators': randint(150, 250),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.03, 0.07),
    'subsample': uniform(0.7, 0.2),
    'colsample_bytree': uniform(0.6, 0.3),
    'lambda': randint(100, 200),
    'alpha': uniform(0.5, 1.0),
    'min_child_weight': randint(10, 20)
}

param_grid_xgb = {
    'n_estimators': randint(150, 250),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.03, 0.07),
    'subsample': uniform(0.7, 0.2),
    'colsample_bytree': uniform(0.6, 0.3),
    'lambda': randint(100, 200),
    'alpha': uniform(0.5, 1.0),
    'min_child_weight': randint(10, 20)
}

param_grid_rf = {
    'n_estimators': randint(50, 400),
    'max_depth': randint(2, 15),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(2, 15)
}

# 使用 KFold 代替 RepeatedKFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 训练模型
best_lgb = train_model_with_tuning(LGBMRegressor(random_state=42), param_grid_lgb, X_train, y_train, cv=kf)
best_xgb = train_model_with_tuning(xgb.XGBRegressor(objective='reg:squarederror', random_state=42), param_grid_xgb, X_train, y_train, cv=kf)
best_rf = train_model_with_tuning(RandomForestRegressor(random_state=42), param_grid_rf, X_train, y_train, cv=kf)

# 6. 模型融合：堆叠融合
# 使用交叉验证的预测结果作为堆叠模型的输入
lgb_preds = cross_val_predict(best_lgb, X_train, y_train, cv=kf)
xgb_preds = cross_val_predict(best_xgb, X_train, y_train, cv=kf)
rf_preds = cross_val_predict(best_rf, X_train, y_train, cv=kf)

X_train_stacked = np.column_stack((lgb_preds, xgb_preds, rf_preds))
X_test_stacked = np.column_stack((
    best_lgb.predict(X_test),
    best_xgb.predict(X_test),
    best_rf.predict(X_test)
))

# 使用 Ridge 作为堆叠模型的最终估计器
stacking_model = StackingRegressor(
    estimators=[
        ('lgb', best_lgb),
        ('xgb', best_xgb),
        ('rf', best_rf)
    ],
    final_estimator=Ridge(alpha=1.0)  # 使用带有正则化的线性模型
)

stacking_model.fit(X_train_stacked, y_train)

# 7. 模型评估
def evaluate_model(model, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_R2 = r2_score(y_train, y_train_pred)
    test_R2 = r2_score(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    return train_R2, test_R2, train_mse, test_mse, train_rmse, test_rmse, train_mae, test_mae

# 定义一个函数来绘制预测值与实测值的散点图
def plot_predictions_vs_actual(y_true, y_pred, model_name):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.6, label="Predictions")
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], color='red', linestyle='--', label="Perfect Fit")
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(f"Predicted vs Actual Values for {model_name}")
    plt.legend()

# 创建 PDF 文件保存所有图片
pdf_path = r"D:\data\Results09.pdf"  # 替换为你想要保存PDF的路径
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)  # 确保目录存在

with PdfPages(pdf_path) as pdf:
    # 保存特征重要性图
    plt.figure(figsize=(10, 6))
    sns.barplot(x=feature_importances, y=abbreviated_feature_names)
    plt.title("Feature Importances")
    plt.tight_layout()  # 调整布局以确保特征名完整显示
    pdf.savefig()  # 保存到 PDF
    plt.close()

    # 评估各个模型并保存预测图
    models = {
        'XGBoost': best_xgb,
        'Random Forest': best_rf,
        'LightGBM': best_lgb,
        'Stacking': stacking_model
    }

    results = {}
    for name, model in models.items():
        if name == 'Stacking':
            train_R2, test_R2, train_mse, test_mse, train_rmse, test_rmse, train_mae, test_mae = evaluate_model(model, X_train_stacked, y_train, X_test_stacked, y_test)
            y_pred = model.predict(X_test_stacked)
        else:
            train_R2, test_R2, train_mse, test_mse, train_rmse, test_rmse, train_mae, test_mae = evaluate_model(model, X_train, y_train, X_test, y_test)
            y_pred = model.predict(X_test)

        results[name] = {
            'Train R2': train_R2,
            'Test R2': test_R2,
            'Train MSE': train_mse,
            'Test MSE': test_mse,
            'Train RMSE': train_rmse,
            'Test RMSE': test_rmse,
            'Train MAE': train_mae,
            'Test MAE': test_mae
        }

        plot_predictions_vs_actual(y_test, y_pred, name)
        pdf.savefig()  # 保存到 PDF
        plt.close()

    # 输出评估参数
    plt.figure(figsize=(10, 6))
    plt.axis('off')  # 关闭坐标轴
    plt.title("Model Evaluation Metrics")
    table_data = [
        ["Model", "Train R2", "Test R2", "Train MSE", "Test MSE", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE"]
    ]
    for name, metrics in results.items():
        table_data.append([
            name,
            f"{metrics['Train R2']:.4f}",
            f"{metrics['Test R2']:.4f}",
            f"{metrics['Train MSE']:.4f}",
            f"{metrics['Test MSE']:.4f}",
            f"{metrics['Train RMSE']:.4f}",
            f"{metrics['Test RMSE']:.4f}",
            f"{metrics['Train MAE']:.4f}",
            f"{metrics['Test MAE']:.4f}"
        ])
    table = plt.table(cellText=table_data, loc='center', cellLoc='center', colLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    pdf.savefig()  # 保存到 PDF
    plt.close()

print(f"All plots and evaluation metrics have been saved to {pdf_path}")

XGBoost version: 2.1.1
Detected encoding: UTF-8-SIG


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:DOI: object